In [1]:
import os
import datetime
import torch
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import DataLoader
import numpy as np
from tqdm.autonotebook import tqdm
from pycocotools.cocoeval import COCOeval

# custom functions
from model import SSD
from dataloader import ObjectDataset, collate_fn
from utils import Encoder, make_dirs, BoxesGen
from utils import SSDAugmentation, Loss


In [2]:
from torch.utils.tensorboard import SummaryWriter
# check cuda availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
classes = ["background", "bicycle", "car", "motorcycle", "bus"] 
colors = [None, (164, 80, 133), (39, 129, 113), (83, 122, 114), (99, 81, 172)]


In [4]:
def train_epoc(modellib, train_loader, epochs, writer, loss_fn, optimizer, scheduler):
    train_loss_l = []
    train_acc_l = []
    for epoch in range(first_epoch, epochs):
        modellib.train()
        total = 0
        sum_loss = 0
        corrects = 0
        num_iter_per_epoch = len(train_loader)
        progress_bar = tqdm(train_loader)
        for i, (img, _, _, grd_box, grd_lbl) in enumerate(progress_bar):
            cnt = grd_lbl.shape[0]
            img = img.cuda()
            grd_box = grd_box.cuda()
            grd_lbl = grd_lbl.cuda()
            modellib.eval()
            pred_box, pred_lbl = modellib(img)
            pred_box, pred_lbl = pred_box.float(), pred_lbl.float()
            # print("grd_box:", grd_box.shape)
            # print("pred_box:", pred_box.shape)
            # print("grd_lbl:", grd_lbl.shape)
            # print("pred_lbl:", pred_lbl.shape)
            grd_box = grd_box.transpose(1, 2).contiguous()
            loss = loss_fn(pred_box, pred_lbl, grd_box, grd_lbl)

            progress_bar.set_description("Epoch: {}. Loss: {:.5f}".format(epoch + 1, loss.item()))
            writer.add_scalar("Train Loss", loss.item(), epoch * num_iter_per_epoch + i)

            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            
            
            total += cnt
            sum_loss += loss.item()
            _, pred = torch.max(pred_lbl, 1)
            corrects += torch.sum(pred.eq(grd_lbl)).item()
        train_loss = sum_loss/total
        train_acc = corrects /total
        evaluate(model, test_loader, epoch, writer, encoder, nms_threshold)
        
        # history
        train_loss_l.append(train_loss)
        train_acc_l.append(train_acc)
        print("Epoch: {}, Train Loss: {}".format(epoch, round(train_loss, 3)))
        
    return modellib, train_loss_l, train_acc_l


In [5]:
def evaluate(modellib, test_loader, epoch, writer, encoder, nms_threshold):
    modellib.eval()
    detections = []
    cat_ids = test_loader.dataset.coco.getCatIds()
    progress_bar = tqdm(test_loader)
    count = 0
    for nbatch, (img, img_id, img_size, grd_box, grd_lbl) in enumerate(progress_bar):
        # print("Parsing batch: {}/{}".format(nbatch, len(test_loader)), end="\r")
        progress_bar.set_description("Validation : {}".format(nbatch))
        img = img.cuda()
        with torch.no_grad():
            # Get predictions
            pred_box, pred_lbl = modellib(img)
            pred_box, pred_lbl = pred_box.float(), pred_lbl.float()
            
            # print("grd_box:", grd_box.shape)
            # print("pred_box:", pred_box.shape)
            # print("grd_lbl:", grd_lbl.shape)
            # print("pred_lbl:", pred_lbl.shape)       
        
            for idx in range(pred_box.shape[0]):
                pred_box_i = pred_box[idx, :, :].unsqueeze(0)
                pred_lbl_i = pred_lbl[idx, :, :].unsqueeze(0)
                try:
                    result = encoder.decode_batch(pred_box_i, pred_lbl_i, nms_threshold, 200)[0]
                except:
                    print("No object detected in idx: {}".format(idx))
                    continue

                height, width = img_size[idx]
                loc, label, prob = [r.cpu().numpy() for r in result]
                for loc_, label_, prob_ in zip(loc, label, prob):
                    detections.append([img_id[idx], loc_[0] * width, loc_[1] * height, (loc_[2] - loc_[0]) * width,
                                       (loc_[3] - loc_[1]) * height, prob_,
                                       cat_ids[label_ - 1]])
        count = count + 1

    detections = np.array(detections, dtype=np.float32)
    coco_eval = COCOeval(test_loader.dataset.coco, test_loader.dataset.coco.loadRes(detections), iouType="bbox")
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()
    checkpoint = {"epoch": epoch, "model_state_dict": model.state_dict(),
                "optimizer": optimizer.state_dict(), 
                "scheduler": scheduler.state_dict()}
    torch.save(checkpoint, checkpoint_path)

    writer.add_scalar("Test mAP", coco_eval.stats[0], epoch)



In [6]:
torch.cuda.manual_seed(123)
batch_size = 1
num_workers = 4
log_dir = "logs"
lr = 2.6e-3
epochs = 10
momentum = 0.9
nms_threshold = 0.5
weight_decay = 0.0005
multi_step = [43, 54]
data_path = "data/vehicles"
save_folder = "trained_models"
date_time = datetime.datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
   
log_path = os.path.join(log_dir, date_time)
save_path = os.path.join(log_path, save_folder)
make_dirs(save_path)
checkpoint_path = os.path.join(save_path, "model_ssd.pth")

train_params = {"batch_size": batch_size,  "shuffle": True,
                "drop_last": False, "num_workers": num_workers,
                "collate_fn": collate_fn}

test_params = {"batch_size": batch_size, "shuffle": False,
                "drop_last": False, "num_workers": num_workers,
                "collate_fn": collate_fn}

In [7]:
dboxes = BoxesGen()
encoder = Encoder(dboxes)

model = SSD(num_classes=4).to(device)
    
train_set = ObjectDataset(data_path, "train", SSDAugmentation(dboxes, (300, 300), val=False))
train_loader = DataLoader(train_set, **train_params)
test_set = ObjectDataset(data_path, "val", SSDAugmentation(dboxes, (300, 300), val=True))
test_loader = DataLoader(test_set, **test_params)

# update lr
lr = lr * (batch_size / 32)

loss_fn = Loss(dboxes).to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum,
                            weight_decay=weight_decay, nesterov=True)

scheduler = MultiStepLR(optimizer = optimizer, milestones = multi_step, gamma=0.1)
# model.cuda()
# loss_fn.cuda()

loading annotations into memory...
Done (t=0.11s)
creating index...
index created!
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


/home/jyothir-dr/miniconda3/envs/detect/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [8]:
if os.path.isfile(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    first_epoch = checkpoint["epoch"] + 1
    # model.module.load_state_dict(checkpoint["model_state_dict"])
    model.load_state_dict(checkpoint["model_state_dict"])

    scheduler.load_state_dict(checkpoint["scheduler"])
    optimizer.load_state_dict(checkpoint["optimizer"])
else:
    first_epoch = 0

In [9]:
writer = SummaryWriter(log_path)

2022-07-19 20:40:07.642938: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [10]:
epochs = 3
modellib, train_loss_l, train_acc_l = train_epoc(model, train_loader, epochs, writer, loss_fn, optimizer, scheduler)

  0%|          | 0/1524 [00:00<?, ?it/s]

  0%|          | 0/344 [00:00<?, ?it/s]

Loading and preparing results...
Converting ndarray to lists...
(68800, 7)
0/68800
DONE (t=0.43s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.36s).
Accumulating evaluation results...
DONE (t=0.52s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.001
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.006
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.013
 Average Recall   

  0%|          | 0/1524 [00:00<?, ?it/s]

KeyboardInterrupt: 